# Extração de Características

Load

In [96]:
# imports
import numpy as np
# from math import prod

%run "at2-preprocessamento.ipynb"

print("\n====================== Importados da at2 ======================")

data, chunks_time, chunks_freq, Pxx = data, chunks_time, chunks_freq, Pxx #pra desbugar o lint

print(f'Data original {data.shape}')

voluntarios, classes_ensaios, canais, janelas, linhas = chunks_time.shape
chunks_time = chunks_time.reshape(voluntarios * classes_ensaios, canais, janelas, linhas)
print(f'Dominio do tempo: {chunks_time.shape} - (voluntario*classes*ensaios, canais, janelas, linhas)')

voluntarios, classes_ensaios, canais, janelas, linhas = chunks_freq.shape
chunks_freq = chunks_freq.reshape(voluntarios * classes_ensaios, canais, janelas, linhas)
print(f'Dominio da frequência:  {chunks_freq.shape} - (voluntario*classes*ensaios, canais, janelas, linhas)')

voluntário, classes_ensaios, canais, janelas = Pxx.shape
Pxx = Pxx.reshape(voluntarios * classes_ensaios, canais, janelas)
print(f'PSD-Welch (dom. da frequência): {Pxx.shape} - (voluntário*classes*ensaios, canais, janelas)')

10, 3, 8, 4, 1600
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (10, 24, 4, 33, 64) - (voluntario, classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (10, 24, 4, 33, 51) - (voluntario, classes+ensaios, canais, janelas, linhas)
Formato (shape) dos dados no domínio da frequência usando o método Welch:
Domínio da frequência: (10, 24, 4, 33) - (voluntário, classes+ensaios, canais, janelas)

====================== Importados da at2 ======================
Data original (10, 24, 4, 1600)
Dominio do tempo: (240, 4, 33, 64) - (voluntario*classes*ensaios, canais, janelas, linhas)
Dominio da frequência:  (240, 4, 33, 51) - (voluntario*classes*ensaios, canais, janelas, linhas)
PSD-Welch (dom. da frequência): (240, 4, 33) - (voluntário*classes*ensaios, canais, janelas)


### Funções de Extração de Características

In [97]:
# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2

##########################################
### FUNÇÕES DE EXTRAÇÃO DE CARACTERÍSTICAS

# Dominio do Tempo
# WAMP, VAR, RMS, WL, ZC
def wamp(x, threshold):
    return np.sum(np.abs(np.diff(x)) > threshold, axis=-1)

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape) - 1))

def wl(x):
    return np.abs(np.diff(x, axis=-1)).sum(axis=-1)

def zc(x):
    return np.sum(np.diff(np.sign(x), axis=-1) != 0, axis=-1)

# Dominio da Frequencia
# FMD, FMN, MMDF, MMNF, PSD
def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def fmn(w):
    # sample_rate = 200
    _,_,_,M = w.shape
    f_j = (np.arange(1, M+1) * 200 )/ (2 * M)  # Frequências correspondentes a cada índice
    weighted_sum = np.sum(f_j * PSD(w))
    mean_freq = weighted_sum / fmd(w)*2
    return mean_freq

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2

def mmnf(w):
    # sample_rate = 200
    _,_,_,M = w.shape
    f_j = np.arange(1, M+1) * 200 / (2 * M) 
    weighted_sum = np.sum(f_j * np.abs(w), axis=-1)
    mean_freq = weighted_sum / mmdf(w)*2
    return mean_freq

#PSD = Pxx ; welch
# f, Pxx = welch(data, fs=200, nperseg=64, noverlap=32)


### Geração do vetor de características

In [98]:
final_data = list()

#dominio do tempo
final_data.append(wamp(chunks_time, 5))
final_data.append(var(chunks_time))
final_data.append(rms(chunks_time))
final_data.append(wl(chunks_time))
final_data.append(zc(chunks_time))

#dominio da freq
final_data.append(fmd(chunks_freq))
final_data.append(fmn(chunks_freq))
final_data.append(mmdf(chunks_freq))
final_data.append(mmnf(chunks_freq))
final_data.append(Pxx)

final = np.array(final_data)
final.shape

(10, 240, 4, 33)